In [ ]:
import httplib2 as lib2 #Example of the "as" function
import google.oauth2.credentials #Importing a sub-package
from google_auth_httplib2 import AuthorizedHttp
import pandas as pd
from googleapiclient.discovery import build as google_build #An example with all the statements together
 #Data processing packages
import boto3 
import numpy
import json
import requests
from datetime import datetime, timedelta #importing multiple sub-packages from one package
import datetime as dt
#Packages needed for connecting with Google API

    
#We have this from previous sections
access_token = access_token
refresh_token = refresh_token
client_id = client_id
client_secret = client_secret

#This is consistent for all Google services
token_uri = 'https://accounts.google.com/o/oauth2/token'

#We are essentially setting the expiry date to 1 day before today, which will make it always expire
token_expiry = dt.datetime.now() - timedelta(days = 1) 

#¯\_(ツ)_/¯
user_agent = 'my-user-agent/1.0'

#The real code that initalized the client
credentials = google.oauth2.credentials.Credentials(access_token,  
                                                    refresh_token=refresh_token,
                                                    token_uri='https://accounts.google.com/o/oauth2/token',
                                                    client_id=client_id,
                                                    client_secret=client_secret)

#Authorize client
authorized = AuthorizedHttp(credentials=credentials)

def refreshToken( client_id, client_secret , refresh_token):
    params = {
            "grant_type": "refresh_token",
            "client_id": client_id ,
            "client_secret": client_secret,
            "refresh_token": refresh_token
    }

    authorization_url = "https://www.googleapis.com/oauth2/v3/token"

    r = requests.post(authorization_url,data=params)


    if r.ok:
         return r.json()['access_token']
    else:
        return None

access_token =  refreshToken( client_id, client_secret , refresh_token)

#API Name and Verison, these don't change until 
#they release a new API version for us to play with. 
api_name = 'analyticsreporting'
api_version = 'v4'

#Let's build the client
api_client = google_build(serviceName=api_name, version=api_version, http=authorized)

#Parse the response of API
def prase_response(report):

    """Parses and prints the Analytics Reporting API V4 response"""
    #Initialize results, in list format because two dataframes might return
    result_list = []

    #Initialize empty data container for the two dateranges (if there are two that is)
    data_csv = []
    data_csv2 = []

    #Initialize header rows
    header_row = []

    #Get column headers, metric headers, and dimension headers.
    columnHeader = report.get('columnHeader', {})
    metricHeaders = columnHeader.get('metricHeader', {}).get('metricHeaderEntries', [])
    dimensionHeaders = columnHeader.get('dimensions', [])

    #Combine all of those headers into the header_row, which is in a list format
    for dheader in dimensionHeaders:
        header_row.append(dheader)
    for mheader in metricHeaders:
        header_row.append(mheader['name'])

    #Get data from each of the rows, and append them into a list
    rows = report.get('data', {}).get('rows', [])
    for row in rows:
        row_temp = []
        dimensions = row.get('dimensions', [])
        metrics = row.get('metrics', [])
        for d in dimensions:
            row_temp.append(d)
        for m in metrics[0]['values']:
            row_temp.append(m)
        data_csv.append(row_temp)

        #In case of a second date range, do the same thing for the second request
        if len(metrics) == 2:
            row_temp2 = []
            for d in dimensions:
                row_temp2.append(d)
            for m in metrics[1]['values']:
                row_temp2.append(m)
            data_csv2.append(row_temp2)

    #Putting those list formats into pandas dataframe, and append them into the final result
    result_df = pd.DataFrame(data_csv, columns=header_row)
    result_list.append(result_df)
    if data_csv2 != []:
        result_list.append(pd.DataFrame(data_csv2, columns=header_row))

    return result_list

In [ ]:
n1 = 1
date_1_days_ago = datetime.now() - timedelta(days=n1)
date_1_days_ago = date_1_days_ago.strftime('%Y-%m-%d')


# -- visitors--

# overview

sample_request = {
  'viewId': viewId,
  'dateRanges': {
  'startDate': date_1_days_ago,
  'endDate': date_1_days_ago
  },
  'dimensions': [{'name':'ga:clientId'},{'name':'ga:date'}],
  'metrics': [{'expression': 'ga:pageviews'},{'expression':'ga:sessions'},{'expression': 'ga:uniquePageviews'},{'expression':'ga:sessionDuration'},{'expression':'ga:bounces'},{'expression':'ga:totalEvents'}]
  }

response = api_client.reports().batchGet(
  body={
  'reportRequests': sample_request
  }).execute()

response_data = response.get('reports', [])[0]
page_view17 = pd.DataFrame(prase_response(response_data)[0])
page_view17['ga:pageviews'] = page_view17['ga:pageviews'].astype(int)
page_view17['ga:%pageviews'] = page_view17['ga:pageviews']/page_view17['ga:pageviews'].sum() * 100
page_view17[['ga:%pageviews']] = page_view17[['ga:%pageviews']].apply(pd.to_numeric)
decimals = pd.Series([2], index=['ga:%pageviews'])
page_view17 = page_view17.round(decimals).sort_values('ga:pageviews',ascending = False).head(10)
page_view17.columns=[i.replace("ga:","") for i in list(page_view17.columns) if "ga:" in i]
page_view17['date'] = pd.to_datetime(page_view17['date'])
new17=page_view17.to_csv(index=False)

# # -- visitors--



# Locations


sample_request = {
  'viewId': viewId,
  'dateRanges': {
  'startDate': date_1_days_ago,
  'endDate': date_1_days_ago
  },
  'dimensions': [{'name': 'ga:country'},{'name': 'ga:city'},{'name': 'ga:region'},{'name':'ga:continent'},{'name':'ga:browser'},{'name':'ga:date'}],
  'metrics': [{'expression':'ga:newUsers'},{'expression':'ga:pageviews'}]
  }

response = api_client.reports().batchGet(
  body={
  'reportRequests': sample_request
  }).execute()

response_data = response.get('reports', [])[0]
df=pd.DataFrame(prase_response(response_data)[0])
df.columns=[i.replace("ga:","") for i in list(df.columns) if "ga:" in i]
df['date'] = pd.to_datetime(df['date'])
new=df.to_csv(index=False)

# Device

sample_request = {
  'viewId': viewId,
  'dateRanges': {
  'startDate': date_1_days_ago,
  'endDate': date_1_days_ago
  },
  'dimensions': [{'name':'ga:mobileDeviceModel'},{'name':'ga:mobileDeviceBranding'},{'name':'ga:deviceCategory'},{'name':'ga:browserSize'},{'name':'ga:browser'},{'name':'ga:date'}],
  'metrics': [{'expression':'ga:newUsers'}]
  }

response = api_client.reports().batchGet(
  body={
  'reportRequests': sample_request
  }).execute()

response_data = response.get('reports', [])[0]
df1=pd.DataFrame(prase_response(response_data)[0])
df1.columns=[i.replace("ga:","") for i in list(df1.columns) if "ga:" in i]
df1['date'] = pd.to_datetime(df1['date'])
new1=df1.to_csv(index=False)

# User_explorer

sample_request = {
  'viewId': viewId,
  'dateRanges': {
  'startDate': date_1_days_ago,
  'endDate': date_1_days_ago
  },
  'dimensions': [{'name':'ga:clientId'},{'name':'ga:date'}],
  'metrics': [{'expression':'ga:sessions'},{'expression': 'ga:transactionRevenue'},{'expression':'ga:sessionDuration'},{'expression':'ga:bouncerate'},{'expression':'ga:transactions'},{'expression':'ga:goalConversionRateAll'}]
  }

response = api_client.reports().batchGet(
  body={
  'reportRequests': sample_request
  }).execute()

response_data = response.get('reports', [])[0]
page_view18 = pd.DataFrame(prase_response(response_data)[0])
page_view18.columns=[i.replace("ga:","") for i in list(page_view18.columns) if "ga:" in i]
page_view18['date'] = pd.to_datetime(page_view18['date'])
new2=page_view18.to_csv(index=False)

# Active Users

sample_request = {
  'viewId': viewId,
  'dateRanges': {
  'startDate': date_1_days_ago,
  'endDate': date_1_days_ago
  },
  'dimensions': [{'name': 'ga:date'}],
  'metrics': [{'expression': 'ga:1dayUsers'}]
  }

response = api_client.reports().batchGet(
  body={
  'reportRequests': sample_request
  }).execute()

response_data = response.get('reports', [])[0]
df10=pd.DataFrame(prase_response(response_data)[0])
df10.columns=[i.replace(":","_") for i in list(df10.columns) if ":" in i]
df10['ga_date'] = pd.to_datetime(df10['ga_date'])

sample_request = {
  'viewId': viewId,
  'dateRanges': {
  'startDate': date_1_days_ago,
  'endDate': date_1_days_ago
  },
  'dimensions': [{'name': 'ga:date'}],
  'metrics': [{'expression': 'ga:7dayUsers'}]
  }

response = api_client.reports().batchGet(
  body={
  'reportRequests': sample_request
  }).execute()

response_data = response.get('reports', [])[0]
df11=pd.DataFrame(prase_response(response_data)[0])
df11.columns=[i.replace(":","_") for i in list(df11.columns) if ":" in i]
df11['ga_date'] = pd.to_datetime(df11['ga_date'])

sample_request = {
  'viewId': viewId,
  'dateRanges': {
  'startDate': date_1_days_ago,
  'endDate': date_1_days_ago
  },
  'dimensions': [{'name': 'ga:date'}],
  'metrics': [{'expression': 'ga:14dayUsers'}]
  }

response = api_client.reports().batchGet(
  body={
  'reportRequests': sample_request
  }).execute()

response_data = response.get('reports', [])[0]
df12=pd.DataFrame(prase_response(response_data)[0])
df12.columns=[i.replace(":","_") for i in list(df12.columns) if ":" in i]
df12['ga_date'] = pd.to_datetime(df12['ga_date'])

sample_request = {
  'viewId': viewId,
  'dateRanges': {
  'startDate': date_1_days_ago,
  'endDate': date_1_days_ago
  },
  'dimensions': [{'name': 'ga:date'}],
  'metrics': [{'expression': 'ga:28dayUsers'}]
  }

response = api_client.reports().batchGet(
  body={
  'reportRequests': sample_request
  }).execute()

response_data = response.get('reports', [])[0]
df13=pd.DataFrame(prase_response(response_data)[0])
df13.columns=[i.replace(":","_") for i in list(df13.columns) if ":" in i]
df13['ga_date'] = pd.to_datetime(df13['ga_date'])

sample_request = {
  'viewId': viewId,
  'dateRanges': {
  'startDate': date_1_days_ago,
  'endDate': date_1_days_ago
  },
  'dimensions': [{'name': 'ga:date'}],
  'metrics': [{'expression': 'ga:30dayUsers'}]
  }

response = api_client.reports().batchGet(
  body={
  'reportRequests': sample_request
  }).execute()

response_data = response.get('reports', [])[0]
df14=pd.DataFrame(prase_response(response_data)[0])
df14.columns=[i.replace(":","_") for i in list(df14.columns) if ":" in i]
df14['ga_date'] = pd.to_datetime(df14['ga_date'])
df_final = pd.merge(pd.merge(pd.merge(pd.merge(df10,df11,on='ga_date'),df12,on='ga_date'),df13,on='ga_date'),df14,on='ga_date')
new3=df_final.to_csv(index=False)


# each_language detail

sample_request = {
  'viewId': viewId,
  'dateRanges': {
  'startDate': date_1_days_ago,
  'endDate': date_1_days_ago
  },
  'dimensions': [{'name': 'ga:language'},{'name':'ga:date'}],
  'metrics': [{'expression': 'ga:users'},{'expression': 'ga:newUsers'},{'expression': 'ga:sessions'},{'expression': 'ga:bounceRate'},{'expression': 'ga:pageviewsPerSession'},{'expression':'ga:avgSessionDuration'}]
  }

response = api_client.reports().batchGet(
  body={
  'reportRequests': sample_request
  }).execute()

response_data = response.get('reports', [])[0]
df15=pd.DataFrame(prase_response(response_data)[0])
df15.columns=[i.replace("ga:","") for i in list(df15.columns) if "ga:" in i]
df15['date'] = pd.to_datetime(df15['date'])
new15=df15.to_csv(index=False)

# new user vs returning

sample_request = {
  'viewId': viewId,
  'dateRanges': {
  'startDate': date_1_days_ago,
  'endDate': date_1_days_ago
  },
  'dimensions': [{'name':'ga:userType'},{'name':'ga:date'}],
  'metrics': [{'expression': 'ga:users'},{'expression': 'ga:newUsers'},{'expression': 'ga:sessions'},{'expression': 'ga:bounceRate'},{'expression': 'ga:pageviewsPerSession'},{'expression':'ga:avgSessionDuration'}]
  }

response = api_client.reports().batchGet(
  body={
  'reportRequests': sample_request
  }).execute()

response_data = response.get('reports', [])[0]
df16=pd.DataFrame(prase_response(response_data)[0])
df16.columns=[i.replace("ga:","") for i in list(df16.columns) if "ga:" in i]
df16['date'] = pd.to_datetime(df16['date'])
new16=df16.to_csv(index=False)

# social network detail
sample_request = {
  'viewId': viewId,
  'dateRanges': {
  'startDate': date_1_days_ago,
  'endDate': date_1_days_ago
  },
  'dimensions': [{'name':'ga:socialNetwork'},{'name':'ga:date'}],
  'metrics': [{'expression': 'ga:users'},{'expression': 'ga:newUsers'},{'expression':'ga:sessions'},{'expression':'ga:pageviews'},{'expression':'ga:percentNewSessions'},{'expression':'ga:bounceRate'},{'expression': 'ga:pageviewsPerSession'},{'expression':'ga:avgSessionDuration'}]
  }

response = api_client.reports().batchGet(
  body={
  'reportRequests': sample_request
  }).execute()

response_data = response.get('reports', [])[0]
df19=pd.DataFrame(prase_response(response_data)[0])
df19.columns=[i.replace("ga:","") for i in list(df19.columns) if "ga:" in i]
df19[['percentNewSessions','bounceRate','pageviewsPerSession','avgSessionDuration']] = df19[['percentNewSessions','bounceRate','pageviewsPerSession','avgSessionDuration']].astype(float)
df19[['percentNewSessions','bounceRate','pageviewsPerSession','avgSessionDuration']] = df19[['percentNewSessions','bounceRate','pageviewsPerSession','avgSessionDuration']].astype(int)
df19 = df19.round({'percentNewSessions':2 ,'bounceRate':2,'pageviewsPerSession':2,'avgSessionDuration':2})
df19['date'] = pd.to_datetime(df19['date'])
new19=df19.to_csv(index=False)

# each_total_user

sample_request = {
  'viewId': viewId,
  'dateRanges': {
  'startDate': date_1_days_ago,
  'endDate': date_1_days_ago
  },
  'dimensions': [{'name':'ga:date'}],
  'metrics': [{'expression': 'ga:users'}]
  }

response = api_client.reports().batchGet( 
  body={
  'reportRequests': sample_request
  }).execute()

response_data = response.get('reports', [])[0]
df23=pd.DataFrame(prase_response(response_data)[0])
df23.columns=[i.replace("ga:","") for i in list(df23.columns) if "ga:" in i]
df23['date'] = pd.to_datetime(df23['date'])
new23=df23.to_csv(index=False)

# # ---Behaviour---

# # -Overview-

# # Page


sample_request = {
  'viewId': viewId,
  'dateRanges': {
  'startDate': date_1_days_ago,
  'endDate': date_1_days_ago
  },
  'dimensions': [{'name':'ga:pagePath'},{'name':'ga:date'}],
  'metrics': [{'expression': 'ga:pageviews'},{'expression': 'ga:uniquepageviews'},{'expression':'ga:avgTimeOnPage'},{'expression':'ga:entrances'},{'expression':'ga:bounceRate'},{'expression':'ga:exitRate'}]
  }

response = api_client.reports().batchGet(
  body={
  'reportRequests': sample_request
  }).execute()

response_data = response.get('reports', [])[0]
page_view6 = pd.DataFrame(prase_response(response_data)[0])
page_view6['ga:pageviews'] = page_view6['ga:pageviews'].astype(int)
page_view6['ga:%pageviews'] = page_view6['ga:pageviews']/page_view6['ga:pageviews'].sum() * 100
page_view6[['ga:avgTimeOnPage','ga:bounceRate','ga:exitRate','ga:%pageviews']] = page_view6[['ga:avgTimeOnPage','ga:bounceRate','ga:exitRate','ga:%pageviews']].apply(pd.to_numeric)
page_view6['ga:avgTimeOnPage'] = page_view6['ga:avgTimeOnPage'].astype(int)
page_view6['ga:avgTimeOnPage'] = [str(dt.timedelta(seconds=i)) for i in page_view6['ga:avgTimeOnPage']]
decimals = pd.Series([2,2,2,2], index=['ga:avgTimeOnPage','ga:bounceRate','ga:exitRate','ga:%pageviews'])
page_view6 = page_view6.round(decimals)
page_view6.columns=[i.replace("ga:","") for i in list(page_view6.columns) if "ga:" in i]
page_view6['date'] = pd.to_datetime(page_view6['date'])
dic6=page_view6.to_csv(index=False)

# Entry Page

sample_request = {
  'viewId': viewId,
  'dateRanges': {
  'startDate': date_1_days_ago,
  'endDate': date_1_days_ago
  },
  'dimensions': [{'name':'ga:landingPagePath'},{'name':'ga:date'}],
  'metrics': [{'expression': 'ga:entrances'},{'expression': 'ga:bounces'},{'expression':'ga:bouncerate'}]
  }

response = api_client.reports().batchGet(
  body={
  'reportRequests': sample_request
  }).execute()

response_data = response.get('reports', [])[0]
page_view171 = pd.DataFrame(prase_response(response_data)[0])
page_view171.columns=[i.replace("ga:","") for i in list(page_view171.columns) if "ga:" in i]
page_view171['bouncerate'] = page_view171['bouncerate'].astype(float)
page_view171['bouncerate'] = page_view171['bouncerate'].astype(int)
page_view171['bouncerate'] = page_view171['bouncerate'].round(2)
page_view171 = page_view171.rename(columns={"landingPagePath": "entrypageurl", "entrances": "entrances","bounces":"bounces","bouncerate":"bouncerate"})
page_view171['date'] = pd.to_datetime(page_view171['date'])
dic7=page_view171.to_csv(index=False)


# # Exit page

sample_request = {
  'viewId': viewId,
  'dateRanges': {
  'startDate': date_1_days_ago,
  'endDate': date_1_days_ago
  },
  'dimensions': [{'name':'ga:exitPagePath'},{'name':'ga:date'}],
  'metrics': [{'expression': 'ga:uniquePageviews'},{'expression': 'ga:exits'},{'expression':'ga:exitRate'}]
  }

response = api_client.reports().batchGet(
  body={
  'reportRequests': sample_request
  }).execute()

response_data = response.get('reports', [])[0]
page_view18 = pd.DataFrame(prase_response(response_data)[0])
decimals = pd.Series([2], index=['ga:exitRate'])
page_view18.columns=[i.replace("ga:","") for i in list(page_view18.columns) if "ga:" in i]
page_view18['exitRate'] = page_view18['exitRate'].astype(float)
page_view18['exitRate'] = page_view18['exitRate'].astype(int)
page_view18['exitRate'] = page_view18['exitRate'].round(2)
page_view18 = page_view18.rename(columns={"landingPagePath": "entrypageurl","uniquePageviews":"uniquePageviews","exits":"exits","exitRate":"exitRate"})
page_view18['date'] = pd.to_datetime(page_view18['date'])
dic17 =page_view18.to_csv(index=False)


# # Page Title

sample_request = {
  'viewId': viewId,
  'dateRanges': {
  'startDate': date_1_days_ago,
  'endDate': date_1_days_ago
  },
  'dimensions': [{'name':'ga:pageTitle'},{'name':'ga:date'}],
  'metrics': [{'expression': 'ga:pageviews'},{'expression': 'ga:uniquepageviews'},{'expression':'ga:avgTimeOnPage'},{'expression':'ga:entrances'},{'expression':'ga:bounceRate'},{'expression':'ga:exitRate'}]
  }

response = api_client.reports().batchGet(
  body={
  'reportRequests': sample_request
  }).execute()

response_data = response.get('reports', [])[0]
page_view5 = pd.DataFrame(prase_response(response_data)[0])
page_view5['ga:pageviews'] = page_view5['ga:pageviews'].astype(int)
page_view5['ga:%pageviews'] = page_view5['ga:pageviews']/page_view5['ga:pageviews'].sum() * 100
page_view5[['ga:avgTimeOnPage','ga:bounceRate','ga:exitRate','ga:%pageviews']] = page_view5[['ga:avgTimeOnPage','ga:bounceRate','ga:exitRate','ga:%pageviews']].apply(pd.to_numeric)
page_view5['ga:avgTimeOnPage'] = page_view5['ga:avgTimeOnPage'].astype(int)
page_view5['ga:avgTimeOnPage'] = [str(dt.timedelta(seconds=i)) for i in page_view5['ga:avgTimeOnPage']]
decimals = pd.Series([2,2,2,2], index=['ga:avgTimeOnPage','ga:bounceRate','ga:exitRate','ga:%pageviews'])
page_view5 = page_view5.round(decimals)
page_view5.columns=[i.replace("ga:","") for i in list(page_view5.columns) if "ga:" in i]
page_view5['date'] = pd.to_datetime(page_view5['date'])
dic5=page_view5.to_csv(index=False)

# # event 

sample_request = {
  'viewId': viewId,
  'dateRanges': {
  'startDate': date_1_days_ago,
  'endDate': date_1_days_ago
  },
  'dimensions': [{'name':'ga:eventAction'},{'name':'ga:eventCategory'},{'name':'ga:eventLabel'},{'name':'ga:date'}],
  'metrics': [{'expression': 'ga:totalEvents'},{'expression':'ga:uniqueEvents'}]
  }

response = api_client.reports().batchGet(
  body={
  'reportRequests': sample_request
  }).execute()

response_data = response.get('reports', [])[0]
page_view2 = pd.DataFrame(prase_response(response_data)[0])
page_view2['ga:totalEvents'] = page_view2['ga:totalEvents'].astype(int)
page_view2['ga:%totalEvents'] = page_view2['ga:totalEvents']/page_view2['ga:totalEvents'].sum() * 100
decimals = pd.Series([2], index=['ga:%totalEvents'])
page_view2 = page_view2.round(decimals)
page_view2.columns=[i.replace("ga:","") for i in list(page_view2.columns) if "ga:" in i]
page_view2['date'] = pd.to_datetime(page_view2['date'])
dic10=page_view2.to_csv(index=False)


##-frequency--

# # visitor 


sample_request = {
  'viewId': viewId,
  'dateRanges': {
  'startDate': date_1_days_ago,
  'endDate': date_1_days_ago
  },
  'dimensions': [{'name':'ga:userType'},{'name':'ga:date'}],
  'metrics': [{'expression': 'ga:pageviews'},{'expression': 'ga:uniquepageviews'},{'expression':'ga:avgTimeOnPage'},{'expression':'ga:entrances'},{'expression':'ga:bounceRate'},{'expression':'ga:exitRate'}]
  }

response = api_client.reports().batchGet(
  body={
  'reportRequests': sample_request
  }).execute()

response_data = response.get('reports', [])[0]
page_view16 = pd.DataFrame(prase_response(response_data)[0])
page_view16['ga:pageviews'] = page_view16['ga:pageviews'].astype(int)
page_view16['ga:%pageviews'] = page_view16['ga:pageviews']/page_view16['ga:pageviews'].sum() * 100
page_view16[['ga:avgTimeOnPage','ga:bounceRate','ga:exitRate','ga:%pageviews']] = page_view16[['ga:avgTimeOnPage','ga:bounceRate','ga:exitRate','ga:%pageviews']].apply(pd.to_numeric)
page_view16['ga:avgTimeOnPage'] = page_view16['ga:avgTimeOnPage'].astype(int)
page_view16['ga:avgTimeOnPage'] = [str(dt.timedelta(seconds=i)) for i in page_view16['ga:avgTimeOnPage']]
decimals = pd.Series([2,2,2], index=['ga:bounceRate','ga:exitRate','ga:%pageviews'])
page_view16 = page_view16.round(decimals).sort_values('ga:pageviews',ascending = False).head(10)
page_view16.columns=[i.replace("ga:","") for i in list(page_view16.columns) if "ga:" in i]
page_view16['date'] = pd.to_datetime(page_view16['date'])
dic16 =page_view16.to_csv(index=False)


# Visitors bucket based on the number of pages viewed and the time they viewed

sample_request = {
  'viewId': viewId,
  'dateRanges': {
  'startDate': date_1_days_ago,
  'endDate': date_1_days_ago
  },
  'dimensions': [{'name':'ga:userType'},{'name':'ga:date'},{'name':'ga:daysSinceLastSession'}],
  'metrics': [{'expression': 'ga:sessions'},{'expression':'ga:pageviews'},{'expression':'ga:avgSessionDuration'},{'expression':'ga:bounceRate'}]
  }


response = api_client.reports().batchGet(
  body={
  'reportRequests': sample_request
  }).execute()
response_data = response.get('reports', [])[0]
page_view21 = pd.DataFrame(prase_response(response_data)[0])
page_view21 = pd.DataFrame(prase_response(response_data)[0])
page_view21['ga:bounceRate'] = page_view21['ga:bounceRate'].apply(pd.to_numeric)
page_view21['ga:avgSessionDuration'] = page_view21['ga:avgSessionDuration'].astype(float)
page_view21['ga:avgSessionDuration'] = page_view21['ga:avgSessionDuration'].astype(int)
page_view21['ga:sessions'] = page_view21['ga:sessions'].astype(int)
page_view21['ga:pageviews'] = page_view21['ga:pageviews'].astype(int)
page_view21['ga:bounceRate'] = page_view21['ga:bounceRate'].astype(float)
page_view21['ga:date'] = pd.to_datetime(page_view21['ga:date'])
page_view21['ga:pageviews/ga:sessions'] = page_view21['ga:pageviews']/page_view21['ga:sessions']
page_view21['ga:avgSessionDuration'] = [str(dt.timedelta(seconds=i)) for i in page_view21['ga:avgSessionDuration']]
page_view21['ga:pageviews/ga:sessions'] = page_view21['ga:pageviews/ga:sessions'].round(2)
decimals = pd.Series(2, index=['ga:bounceRate'])
page_view21 = page_view21.round(decimals)
page_views = []
for i in page_view21['ga:pageviews']:
    if i == 1:
        page_views.append('1 page')
    elif i == 2:
        page_views.append('2 page')
    elif i == 3:
        page_views.append('3 page')
    elif i == 4:
        page_views.append('4 page')
    elif i == 5:
        page_views.append('5 page')
    elif (i >= 6 and i<= 7):
        page_views.append('6-7 page')
    elif (i >= 8 and i<= 10):
        page_views.append('8-10 page')
    elif (i >= 11 and i<= 14):
        page_views.append('11-14 page')
    elif (i >= 15 and i<= 20):
        page_views.append('15-20 page')
    elif i >= 21:
        page_views.append('21+ page')

page_view21['ga:page_views'] = page_views
page_view21['ga:time'] = page_view21['ga:avgSessionDuration'].str.split(':').apply(lambda x: int(x[0]) * 60 + int(x[1])* 60 + int(x[2]))
visit_time = []
for i in page_view21['ga:time']:
    if i <= 10:
        visit_time.append('0-10s')
    elif i > 10 and i <= 30:
        visit_time.append('11-30s')
    elif i > 30 and i <= 60:
        visit_time.append('31-60s')
    elif i > 60 and i <= 120:
        visit_time.append('1-2min')
    elif i > 120 and i <= 240:
        visit_time.append('2-4min')
    elif i > 240 and i<= 420:
        visit_time.append('4-7min') 
    elif i > 420 and i<= 600:
        visit_time.append('7-10min')
    elif i > 600 and i<= 900:
        visit_time.append('10-15min')
    elif i > 900 and i<= 1800:
        visit_time.append('15-30min')
    elif i > 1800:
        visit_time.append('30+min')

page_view21['ga:visit_time'] = visit_time
page_view21.columns=[i.replace("ga:","") for i in list(page_view21.columns) if "ga:" in i]
dic25 =page_view21.to_csv(index=False)

# User Flow

# Source Behaviour Flow

sample_request = {
  'viewId': viewId,
  'pageSize': 10000,
  'dateRanges': {
  'startDate': date_1_days_ago,
  'endDate': date_1_days_ago,

  },
  'dimensions': [{'name': 'ga:clientId'},{'name':'ga:source'},{'name':'ga:dateHourMinute'},{'name':'ga:pageTitle'}],
  'metrics': [{'expression':'ga:sessions'},{'expression':'ga:pageviews'}],

  }


response = api_client.reports().batchGet(
  body={
  'reportRequests': sample_request
  }).execute()

response_data = response.get('reports', [])[0]
page_view11 = pd.DataFrame(prase_response(response_data)[0])
source_df=page_view11

source_df['ga:sessions'] = source_df['ga:sessions'].astype(int)
source_df['ga:dateHourMinute'] = pd.to_datetime(source_df['ga:dateHourMinute'])
source_flow = []
source = []
dates = []
groups=source_df.groupby('ga:clientId')
for client in source_df['ga:clientId'].unique():
    group=groups.get_group(client)
    for row in group.iterrows():
        if row[1]['ga:sessions']==1:
            source.append(row[1]['ga:source'])
            dates.append(row[1]['ga:dateHourMinute'])
            end = row[1]['ga:dateHourMinute'] + timedelta(minutes=30)
            start_date=row[1]['ga:dateHourMinute']
            df = source_df[source_df['ga:clientId']== row[1]['ga:clientId']]
            x = []
            a = df.loc[(df['ga:dateHourMinute'] >= start_date) & (df['ga:dateHourMinute'] <= end)]
            previous_pagetitle=None
            for row in a.iterrows():
                if previous_pagetitle==None:
                    previous_pagetitle=row[1]['ga:pageTitle']
                    x.append(row[1]['ga:pageTitle'])
                elif previous_pagetitle==row[1]['ga:pageTitle']:
                      continue
                elif previous_pagetitle!=row[1]['ga:pageTitle']:
                    previous_pagetitle=row[1]['ga:pageTitle']
                    x.append(row[1]['ga:pageTitle'])
                source_flow.append(x)

source_behvflow = pd.DataFrame({'dates':dates,'source':source,'source_flow':source_flow})
source_behvflow['source_flow'] = source_behvflow["source_flow"].str.join(">") 
source_behvflow['source'] = source_behvflow['source']+'>'+source_behvflow['source_flow']
source_behvflow = source_behvflow.drop(['source_flow'],axis=1)
dic19 =source_behvflow.to_csv(index=False)


# Medium Behaviour Flow

sample_request = {
  'viewId': viewId,
  'pageSize': 10000,
  'dateRanges': {
  'startDate': date_1_days_ago,
  'endDate': date_1_days_ago,

  },
  'dimensions': [{'name': 'ga:clientId'},{'name':'ga:medium'},{'name':'ga:dateHourMinute'},{'name':'ga:pageTitle'}],
  'metrics': [{'expression':'ga:sessions'},{'expression':'ga:pageviews'}],

  }


response = api_client.reports().batchGet(
  body={
  'reportRequests': sample_request
  }).execute()

response_data = response.get('reports', [])[0]
page_view12 = pd.DataFrame(prase_response(response_data)[0])
medium_df=page_view12

medium_df['ga:sessions'] = medium_df['ga:sessions'].astype(int)
medium_df['ga:dateHourMinute'] = pd.to_datetime(medium_df['ga:dateHourMinute'])
medium_flow = []
medium = []
dates = []
groups=medium_df.groupby('ga:clientId')
for client in medium_df['ga:clientId'].unique():
    group=groups.get_group(client)
    for row in group.iterrows():
        if row[1]['ga:sessions']==1:
            medium.append(row[1]['ga:medium'])
            dates.append(row[1]['ga:dateHourMinute'])
            end = row[1]['ga:dateHourMinute'] + timedelta(minutes=30)
            start_date=row[1]['ga:dateHourMinute']
            df = medium_df[medium_df['ga:clientId']== row[1]['ga:clientId']]
            x = []
            a = df.loc[(df['ga:dateHourMinute'] >= start_date) & (df['ga:dateHourMinute'] <= end)]
            previous_pagetitle=None
            for row in a.iterrows():
                if previous_pagetitle==None:
                    previous_pagetitle=row[1]['ga:pageTitle']
                    x.append(row[1]['ga:pageTitle'])
                elif previous_pagetitle==row[1]['ga:pageTitle']:
                    continue
                elif previous_pagetitle!=row[1]['ga:pageTitle']:
                    previous_pagetitle=row[1]['ga:pageTitle']
                    x.append(row[1]['ga:pageTitle'])
        medium_flow.append(x)

medium_behvflow = pd.DataFrame({'dates':dates,'medium':medium,'medium_flow':medium_flow})
medium_behvflow['medium_flow'] = medium_behvflow["medium_flow"].str.join(">") 
medium_behvflow['medium'] = medium_behvflow['medium']+'>'+medium_behvflow['medium_flow']
medium_behvflow = medium_behvflow.drop(['medium_flow'],axis=1)
dic22 =medium_behvflow.to_csv(index=False)

# Landing Page Flow

sample_request = {
  'viewId': viewId,
  'pageSize': 10000,
  'dateRanges': {
  'startDate': date_1_days_ago,
  'endDate': date_1_days_ago,

  },
  'dimensions': [{'name': 'ga:clientId'},{'name':'ga:landingPagePath'},{'name':'ga:dateHourMinute'},{'name':'ga:pageTitle'}],
  'metrics': [{'expression':'ga:sessions'},{'expression':'ga:pageviews'}],

  }


response = api_client.reports().batchGet(
  body={
  'reportRequests': sample_request
  }).execute()

response_data = response.get('reports', [])[0]
page_view12 = pd.DataFrame(prase_response(response_data)[0])
landingPagePath_df=page_view12

landingPagePath_df['ga:sessions'] = landingPagePath_df['ga:sessions'].astype(int)
landingPagePath_df['ga:dateHourMinute'] = pd.to_datetime(landingPagePath_df['ga:dateHourMinute'])
landingPagePath_flow = []
landingPagePath = []
dates = []
groups=landingPagePath_df.groupby('ga:clientId')
for client in landingPagePath_df['ga:clientId'].unique():
    group=groups.get_group(client)
    for row in group.iterrows():
        if row[1]['ga:sessions']==1:
            landingPagePath.append(row[1]['ga:landingPagePath'])
            dates.append(row[1]['ga:dateHourMinute'])
            end = row[1]['ga:dateHourMinute'] + timedelta(minutes=30)
            start_date=row[1]['ga:dateHourMinute']
            df = landingPagePath_df[landingPagePath_df['ga:clientId']== row[1]['ga:clientId']]
            x = []
            a = df.loc[(df['ga:dateHourMinute'] >= start_date) & (df['ga:dateHourMinute'] <= end)]
            previous_pagetitle=None
            for row in a.iterrows():
                if previous_pagetitle==None:
                    previous_pagetitle=row[1]['ga:pageTitle']
                    x.append(row[1]['ga:pageTitle'])
                elif previous_pagetitle==row[1]['ga:pageTitle']:
                    continue
                elif previous_pagetitle!=row[1]['ga:pageTitle']:
                    previous_pagetitle=row[1]['ga:pageTitle']
                    x.append(row[1]['ga:pageTitle'])
        landingPagePath_flow.append(x)


landingPagePath_behvflow = pd.DataFrame({'dates':dates,'landingPagePath':landingPagePath,'landingPagePath_flow':landingPagePath_flow})
landingPagePath_behvflow['landingPagePath_flow'] = landingPagePath_behvflow["landingPagePath_flow"].str.join(">") 
landingPagePath_behvflow['landingPagePath'] = landingPagePath_behvflow['landingPagePath']+'>'+landingPagePath_behvflow['landingPagePath_flow']
landingPagePath_behvflow = landingPagePath_behvflow.drop(['landingPagePath_flow'],axis=1)
dic24 =landingPagePath_behvflow.to_csv(index=False)


# ----Acquisition----  

# overview

sample_request = {
  'viewId': viewId,
  'dateRanges': {
  'startDate': date_1_days_ago,
  'endDate': date_1_days_ago
  },
  'dimensions': [{'name':'ga:landingPagePath'},{'name':'ga:clientId'},{'name':'ga:date'},{'name':'ga:socialNetwork'},{'name':'ga:campaign'},{'name':'ga:source'},{'name':'ga:medium'},{'name':'ga:keyword'},{'name':'ga:referralPath'}],
  'metrics': [{'expression': 'ga:users'}]
  }

response = api_client.reports().batchGet(
  body={
  'reportRequests': sample_request
  }).execute()

response_data = response.get('reports', [])[0]
page_view170 = pd.DataFrame(prase_response(response_data)[0])
page_view170.columns=[i.replace("ga:","") for i in list(page_view170.columns) if "ga:" in i]
page_view170['date'] = pd.to_datetime(page_view170['date'])
page_view170['date'] = pd.to_datetime(page_view170['date'])
dic25 = page_view170.to_csv(index=False)


# channel

sample_request = {
  'viewId': viewId,
  'dateRanges': {
  'startDate': date_1_days_ago,
  'endDate': date_1_days_ago
  },
  'dimensions': [{'name':'ga:channelGrouping'},{'name':'ga:date'}],
  'metrics': [{'expression': 'ga:pageviews'},{'expression':'ga:avgSessionDuration'},{'expression':'ga:entrances'},{'expression':'ga:bounceRate'}]
  }

response = api_client.reports().batchGet(
  body={
  'reportRequests': sample_request
  }).execute()

response_data = response.get('reports', [])[0]
page_view7 = pd.DataFrame(prase_response(response_data)[0])
page_view7['ga:pageviews'] = page_view7['ga:pageviews'].astype(int)
page_view7['ga:entrances'] = page_view7['ga:entrances'].astype(int)
page_view7[['ga:avgSessionDuration','ga:bounceRate']] = page_view7[['ga:avgSessionDuration','ga:bounceRate']].apply(pd.to_numeric)
page_view7['ga:avgSessionDuration'] = page_view7['ga:avgSessionDuration'].astype(int)
page_view7['ga:avgSessionDuration'] = [str(dt.timedelta(seconds=i)) for i in page_view7['ga:avgSessionDuration']]
decimals = pd.Series([2,2], index=['ga:avgSessionDuration','ga:bounceRate'])
page_view7 = page_view7.round(decimals)
page_view7.columns=[i.replace("ga:","") for i in list(page_view7.columns) if "ga:" in i]
page_view7['actionpervisit'] = page_view7['pageviews']/page_view7['entrances']
pae_view7 = page_view7.rename(columns={"channelGrouping":"channel","entrances":"visits","actionpervisit":"actionpervisit","avgSessionDuration":"avgTimeOnwebsite","bounceRate":"bouncerate"})
page_view7['date'] = pd.to_datetime(page_view7['date'])
dic26 = page_view7.to_csv(index=False)


# Refrrer

sample_request = {
  'viewId': viewId,
  'dateRanges': {
  'startDate': date_1_days_ago,
  'endDate': date_1_days_ago
  },
  'dimensions': [{'name':'ga:fullReferrer'},{'name':'ga:date'}],
  'metrics': [{'expression': 'ga:pageviews'},{'expression':'ga:avgSessionDuration'},{'expression':'ga:entrances'},{'expression':'ga:bounceRate'}]
  }

response = api_client.reports().batchGet(
  body={
  'reportRequests': sample_request
  }).execute()

response_data = response.get('reports', [])[0]
page_view8 = pd.DataFrame(prase_response(response_data)[0])
page_view8['ga:pageviews'] = page_view8['ga:pageviews'].astype(int)
page_view8['ga:entrances'] = page_view8['ga:entrances'].astype(int)
page_view8[['ga:avgSessionDuration','ga:bounceRate']] = page_view8[['ga:avgSessionDuration','ga:bounceRate']].apply(pd.to_numeric)
page_view8['ga:avgSessionDuration'] = page_view8['ga:avgSessionDuration'].astype(int)
page_view8['ga:avgSessionDuration'] = [str(dt.timedelta(seconds=i)) for i in page_view8['ga:avgSessionDuration']]
decimals = pd.Series([2,2], index=['ga:avgSessionDuration','ga:bounceRate'])
page_view8 = page_view8.round(decimals)
page_view8.columns=[i.replace("ga:","") for i in list(page_view8.columns) if "ga:" in i]
page_view8['actionpervisit'] = page_view8['pageviews']/page_view8['entrances']
page_view8 = page_view8.rename(columns={"channelGrouping":"channel","entrances":"visits","actionpervisit":"actionpervisit","avgSessionDuration":"avgTimeOnwebsite","bounceRate":"bouncerate"})
page_view8['date'] = pd.to_datetime(page_view8['date'])
dic27 = page_view8.to_csv(index=False)


# # Page details who visited through any Source


sample_request = {
  'viewId': viewId,
  'dateRanges': {
  'startDate': date_1_days_ago,
  'endDate': date_1_days_ago
  },
  'dimensions': [{'name':'ga:source'},{'name':'ga:date'}],
  'metrics': [{'expression': 'ga:pageviews'},{'expression': 'ga:uniquepageviews'},{'expression':'ga:avgTimeOnPage'},{'expression':'ga:entrances'},{'expression':'ga:bounceRate'},{'expression':'ga:exitRate'}]
  }

response = api_client.reports().batchGet(
  body={
  'reportRequests': sample_request
  }).execute()

response_data = response.get('reports', [])[0]
page_view7 = pd.DataFrame(prase_response(response_data)[0])
page_view7['ga:pageviews'] = page_view7['ga:pageviews'].astype(int)
page_view7['ga:%pageviews'] = page_view7['ga:pageviews']/page_view7['ga:pageviews'].sum() * 100
page_view7[['ga:avgTimeOnPage','ga:bounceRate','ga:exitRate','ga:%pageviews']] = page_view7[['ga:avgTimeOnPage','ga:bounceRate','ga:exitRate','ga:%pageviews']].apply(pd.to_numeric)
page_view7['ga:avgTimeOnPage'] = page_view7['ga:avgTimeOnPage'].astype(int)
page_view7['ga:avgTimeOnPage'] = [str(dt.timedelta(seconds=i)) for i in page_view7['ga:avgTimeOnPage']]
decimals = pd.Series([2,2,2,2], index=['ga:avgTimeOnPage','ga:bounceRate','ga:exitRate','ga:%pageviews'])
page_view7 = page_view7.round(decimals)
page_view7.columns=[i.replace("ga:","") for i in list(page_view7.columns) if "ga:" in i]
page_view7['date'] = pd.to_datetime(page_view7['date'])
dic77=page_view7.to_csv(index=False)


# # Page details who are coming from any medium

# In[125]:


sample_request = {
  'viewId': viewId,
  'dateRanges': {
  'startDate': date_1_days_ago,
  'endDate': date_1_days_ago
  },
  'dimensions': [{'name':'ga:medium'},{'name':'ga:date'}],
  'metrics': [{'expression': 'ga:pageviews'},{'expression': 'ga:uniquepageviews'},{'expression':'ga:avgTimeOnPage'},{'expression':'ga:entrances'},{'expression':'ga:bounceRate'},{'expression':'ga:exitRate'}]
  }

response = api_client.reports().batchGet(
  body={
  'reportRequests': sample_request
  }).execute()

response_data = response.get('reports', [])[0]
page_view8 = pd.DataFrame(prase_response(response_data)[0])
page_view8['ga:pageviews'] = page_view8['ga:pageviews'].astype(int)
page_view8['ga:%pageviews'] = page_view8['ga:pageviews']/page_view8['ga:pageviews'].sum() * 100
page_view8[['ga:avgTimeOnPage','ga:bounceRate','ga:exitRate','ga:%pageviews']] = page_view8[['ga:avgTimeOnPage','ga:bounceRate','ga:exitRate','ga:%pageviews']].apply(pd.to_numeric)
page_view8['ga:avgTimeOnPage'] = page_view8['ga:avgTimeOnPage'].astype(int)
page_view8['ga:avgTimeOnPage'] = [str(dt.timedelta(seconds=i)) for i in page_view8['ga:avgTimeOnPage']]
decimals = pd.Series([2,2,2,2], index=['ga:avgTimeOnPage','ga:bounceRate','ga:exitRate','ga:%pageviews'])
page_view8 = page_view8.round(decimals)
page_view8.columns=[i.replace("ga:","") for i in list(page_view8.columns) if "ga:" in i]
page_view8['date'] = pd.to_datetime(page_view8['date'])
dic8=page_view8.to_csv(index=False)

# demographic and tech

sample_request = {
  'viewId': viewId,
  'dateRanges': {
  'startDate': date_1_days_ago,
  'endDate': date_1_days_ago
  },
  'dimensions': [{'name':'ga:city'},{'name':'ga:country'},{'name':'ga:continent'},{'name':'ga:operatingSystem'},{'name':'ga:browser'},{'name':'ga:hostname'},{'name':'ga:date'}],
  'metrics': [{'expression': 'ga:pageviews'},{'expression': 'ga:uniquepageviews'},{'expression':'ga:avgTimeOnPage'},{'expression':'ga:entrances'},{'expression':'ga:bounceRate'},{'expression':'ga:exitRate'}]
  }

response = api_client.reports().batchGet(
  body={
  'reportRequests': sample_request
  }).execute()

response_data = response.get('reports', [])[0]
page_view13 = pd.DataFrame(prase_response(response_data)[0])
page_view13['ga:pageviews'] = page_view13['ga:pageviews'].astype(int)
page_view13['ga:%pageviews'] = page_view13['ga:pageviews']/page_view13['ga:pageviews'].sum() * 100
page_view13[['ga:avgTimeOnPage','ga:bounceRate','ga:exitRate','ga:%pageviews']] = page_view13[['ga:avgTimeOnPage','ga:bounceRate','ga:exitRate','ga:%pageviews']].apply(pd.to_numeric)
page_view13['ga:avgTimeOnPage'] = page_view13['ga:avgTimeOnPage'].astype(int)
page_view13['ga:avgTimeOnPage'] = [str(dt.timedelta(seconds=i)) for i in page_view13['ga:avgTimeOnPage']]
decimals = pd.Series([2,2,2,2], index=['ga:avgTimeOnPage','ga:bounceRate','ga:exitRate','ga:%pageviews'])
page_view13 = page_view13.round(decimals).sort_values('ga:pageviews',ascending = False).head(10)
page_view13 .columns=[i.replace("ga:","") for i in list(page_view13.columns) if "ga:" in i]
page_view13['date'] = pd.to_datetime(page_view13['date'])
dic99=page_view13.to_csv(index=False)

# # graph per day
# # metric vs metric

sample_request = {
  'viewId': viewId,
  'dateRanges': {
  'startDate': date_1_days_ago,
  'endDate': date_1_days_ago
  },
  'dimensions': [{'name':'ga:date'}],
  'metrics': [{'expression': 'ga:pageviews'},{'expression':'ga:uniquePageviews'},{'expression':'ga:avgTimeOnPage'},{'expression':'ga:bounceRate'},{'expression':'ga:exitRate'}]
  }

response = api_client.reports().batchGet(
  body={
  'reportRequests': sample_request
  }).execute()

response_data = response.get('reports', [])[0]
page_view18 = pd.DataFrame(prase_response(response_data)[0])
page_view18['ga:pageviews'] = page_view18['ga:pageviews'].astype(int)
page_view18['ga:%pageviews'] = page_view18['ga:pageviews']/page_view18['ga:pageviews'].sum() * 100
page_view18['ga:date'] = pd.to_datetime(page_view18['ga:date'])
page_view18[['ga:avgTimeOnPage','ga:bounceRate','ga:exitRate','ga:%pageviews']] = page_view18[['ga:avgTimeOnPage','ga:bounceRate','ga:exitRate','ga:%pageviews']].astype(float)
page_view18[['ga:avgTimeOnPage','ga:bounceRate','ga:exitRate','ga:%pageviews']] = page_view18[['ga:avgTimeOnPage','ga:bounceRate','ga:exitRate','ga:%pageviews']].round(2)
page_view18['ga:avgTimeOnPage'] = page_view18['ga:avgTimeOnPage'].astype(int)
page_view18['ga:avgTimeOnPage'] = [str(dt.timedelta(seconds=i)) for i in page_view18['ga:avgTimeOnPage']]
page_view18.columns=[i.replace("ga:","") for i in list(page_view18.columns) if "ga:" in i]
dic18 =page_view18.to_csv(index=False)

# exit links
sample_request = {
  'viewId': viewId,
  'dateRanges': {
  'startDate': date_1_days_ago,
  'endDate': date_1_days_ago
  },
  'dimensions': [{'name':'ga:eventLabel'},{'name':'ga:date'}],
  'metrics': [{'expression': 'ga:totalEvents'}]
  }

response = api_client.reports().batchGet(
  body={
  'reportRequests': sample_request
  }).execute()

response_data = response.get('reports', [])[0]
page_view17 = pd.DataFrame(prase_response(response_data)[0])
page_view17 = page_view17.sort_values('ga:totalEvents',ascending=False)
page_view17.columns=[i.replace("ga:","") for i in list(page_view17.columns) if "ga:" in i]
page_view17['date'] = pd.to_datetime(page_view17['date'])
dic19 =page_view17.to_csv(index=False)